In [9]:
# =========================================================
# SPORTS LEAGUE OPTIMIZATION USING A GENETIC ALGORITHM
# =========================================================

import random
import pandas as pd
import numpy as np
from copy import deepcopy

# ======================
# CONFIGURAÇÕES DO PROBLEMA
# ======================
TEAM_SIZE = 7
NUM_TEAMS = 5
BUDGET_LIMIT = 750
TEAM_STRUCTURE = {"GK": 1, "DEF": 2, "MID": 2, "FWD": 2}
POPULATION_SIZE = 10
TOURNAMENT_SIZE = 3

# ======================
# CARREGAR DADOS
# ======================
df = pd.read_csv("data/players.csv")
df = df.drop(columns=['Unnamed: 0'])

# Agrupar jogadores por posição
players_by_position = {
    "GK": df[df['Position'] == "GK"].to_dict('records'),
    "DEF": df[df['Position'] == "DEF"].to_dict('records'),
    "MID": df[df['Position'] == "MID"].to_dict('records'),
    "FWD": df[df['Position'] == "FWD"].to_dict('records')
}

# ======================
# GERAR EQUIPA VÁLIDA
# ======================
def create_valid_team(available_players):
    team = []
    used_ids = set()
    for pos, count in TEAM_STRUCTURE.items():
        candidates = [p for p in available_players[pos] if id(p) not in used_ids]
        if len(candidates) < count:
            return None, set()  # não há jogadores suficientes
        selected = random.sample(candidates, count)
        for player in selected:
            used_ids.add(id(player))
            team.append(player)
    return team, used_ids

# ======================
# GERAR INDIVÍDUO (LIGA)
# ======================
def create_individual():
    all_players = deepcopy(players_by_position)
    individual = []
    used_ids_total = set()
    for _ in range(NUM_TEAMS):
        team, used_ids = create_valid_team(all_players)
        if team is None or sum(p['Salary (€M)'] for p in team) > BUDGET_LIMIT:
            return None  # equipa inválida
        individual.append(team)
        used_ids_total.update(used_ids)
        for pos in players_by_position:
            all_players[pos] = [p for p in all_players[pos] if id(p) not in used_ids_total]
    return individual

# ======================
# VALIDAR EQUIPA
# ======================
def is_valid_team(team):
    if len(team) != TEAM_SIZE:
        return False
    pos_counts = {"GK": 0, "DEF": 0, "MID": 0, "FWD": 0}
    total_salary = 0
    ids = set()
    for p in team:
        pos_counts[p['Position']] += 1
        total_salary += p['Salary (€M)']
        ids.add(p['Name'])
    return pos_counts == TEAM_STRUCTURE and total_salary <= BUDGET_LIMIT and len(ids) == TEAM_SIZE

# ======================
# FITNESS FUNCTION
# ======================
def fitness(individual):
    if individual is None:
        return float('inf')
    means = []
    used_names = set()
    for team in individual:
        if not is_valid_team(team):
            return float('inf')
        for p in team:
            if p['Name'] in used_names:
                return float('inf')  # jogador repetido
            used_names.add(p['Name'])
        avg_skill = np.mean([p['Skill'] for p in team])
        means.append(avg_skill)
    return np.std(means)  # quanto menor, melhor

# ======================
# GERAR POPULAÇÃO INICIAL
# ======================
def generate_initial_population(size=POPULATION_SIZE):
    population = []
    while len(population) < size:
        indiv = create_individual()
        if indiv is not None:
            population.append(indiv)
    return population

# ======================
# TOURNAMENT SELECTION
# ======================
def tournament_selection(population, k=TOURNAMENT_SIZE):
    competitors = random.sample(population, k)
    best = min(competitors, key=fitness)
    return deepcopy(best)


In [10]:
# ======================
# CROSSOVER BY TEAMS (Corrected version)
# ======================
def crossover_por_equipas(pai1, pai2):
    corte = random.randint(1, NUM_TEAMS - 1)

    def build_child(p1, p2):
        child = []
        used_names = set()

        # Copy first part from parent 1
        for team in p1[:corte]:
            child.append(team)
            used_names.update(p['Name'] for p in team)

        # Add teams from parent 2 only if all players are unique
        for team in p2:
            names = [p['Name'] for p in team]
            if len(child) < NUM_TEAMS:
                if all(name not in used_names for name in names):
                    child.append(team)
                    used_names.update(names)

        # If needed, generate new valid teams to fill remaining slots
        while len(child) < NUM_TEAMS:
            new_team, _ = create_valid_team(players_by_position)
            if new_team and all(p['Name'] not in used_names for p in new_team):
                if sum(p['Salary (€M)'] for p in new_team) <= BUDGET_LIMIT:
                    child.append(new_team)
                    used_names.update(p['Name'] for p in new_team)

        return child

    child1 = build_child(pai1, pai2)
    child2 = build_child(pai2, pai1)
    return child1, child2


In [11]:
# ======================
# CROSSOVER TEST BLOCK
# ======================

# Generate two valid parents
parent1 = create_individual()
while parent1 is None:
    parent1 = create_individual()

parent2 = create_individual()
while parent2 is None:
    parent2 = create_individual()

# Apply crossover
child1, child2 = crossover_por_equipas(parent1, parent2)

# Function to summarize each individual
def summarize_individual(indiv, label):
    avg_skills = [np.mean([p['Skill'] for p in team]) for team in indiv]
    return {
        "Label": label,
        "Avg Skill": round(np.mean(avg_skills), 2),
        "Skill Std Dev": round(np.std(avg_skills), 2),
        "Team Budgets": [round(sum(p['Salary (€M)'] for p in team), 1) for team in indiv],
        "Unique Players": len(set(p['Name'] for team in indiv for p in team))
    }

# Show summary
summary = [
    summarize_individual(parent1, "Parent 1"),
    summarize_individual(parent2, "Parent 2"),
    summarize_individual(child1, "Child 1"),
    summarize_individual(child2, "Child 2"),
]

# Print summary
for s in summary:
    print(f"\n{s['Label']}")
    print(f"  Avg Skill: {s['Avg Skill']}")
    print(f"  Skill Std Dev: {s['Skill Std Dev']}")
    print(f"  Unique Players: {s['Unique Players']}")
    print(f"  Team Budgets: {s['Team Budgets']}")




Parent 1
  Avg Skill: 86.4
  Skill Std Dev: 0.57
  Unique Players: 35
  Team Budgets: [680, 710, 695, 685, 654]

Parent 2
  Avg Skill: 86.4
  Skill Std Dev: 0.85
  Unique Players: 35
  Team Budgets: [677, 725, 647, 705, 670]

Child 1
  Avg Skill: 86.4
  Skill Std Dev: 0.73
  Unique Players: 35
  Team Budgets: [680, 710, 657, 692, 685]

Child 2
  Avg Skill: 86.4
  Skill Std Dev: 1.31
  Unique Players: 35
  Team Budgets: [677, 725, 690, 602, 730]


In [12]:
# ======================
# DEBUG CROSSOVER: SHOW TEAMS BEFORE AND AFTER
# ======================

def print_teams(individual, label):
    print(f"\n{label}")
    for i, team in enumerate(individual):
        print(f"  Team {i+1}:")
        for player in team:
            print(f"    {player['Name']} ({player['Position']}) - Skill: {player['Skill']} - Salary: €{player['Salary (€M)']}M")

# Print teams from both parents
print("="*60)
print_teams(parent1, "PARENT 1 (before crossover)")
print("="*60)
print_teams(parent2, "PARENT 2 (before crossover)")
print("="*60)

# Print teams from both children
print_teams(child1, "CHILD 1 (after crossover)")
print("="*60)
print_teams(child2, "CHILD 2 (after crossover)")
print("="*60)



PARENT 1 (before crossover)
  Team 1:
    Ryan Mitchell (GK) - Skill: 83 - Salary: €85M
    Ethan Howard (DEF) - Skill: 80 - Salary: €70M
    Caleb Fisher (DEF) - Skill: 84 - Salary: €85M
    Bentley Rivera (MID) - Skill: 88 - Salary: €100M
    Austin Torres (MID) - Skill: 82 - Salary: €80M
    Xavier Bryant (FWD) - Skill: 90 - Salary: €120M
    Elijah Sanders (FWD) - Skill: 93 - Salary: €140M
  Team 2:
    Alex Carter (GK) - Skill: 85 - Salary: €90M
    Lucas Bennett (DEF) - Skill: 85 - Salary: €90M
    Owen Parker (DEF) - Skill: 88 - Salary: €100M
    Spencer Ward (MID) - Skill: 84 - Salary: €85M
    Nathan Wright (MID) - Skill: 92 - Salary: €120M
    Chase Murphy (FWD) - Skill: 86 - Salary: €95M
    Julian Scott (FWD) - Skill: 92 - Salary: €130M
  Team 3:
    Chris Thompson (GK) - Skill: 80 - Salary: €80M
    Logan Brooks (DEF) - Skill: 86 - Salary: €95M
    Mason Reed (DEF) - Skill: 82 - Salary: €75M
    Ashton Phillips (MID) - Skill: 90 - Salary: €110M
    Dylan Morgan (MID) - Sk

In [13]:
# Testar geração da população
pop = generate_initial_population(5)  # Testar com 5 para ser mais rápido

for i, indiv in enumerate(pop):
    print(f"\nIndivíduo {i+1}")
    team_means = []
    valid = True
    all_players = set()
    
    for j, team in enumerate(indiv):
        names = [p['Name'] for p in team]
        positions = [p['Position'] for p in team]
        salary = sum(p['Salary (€M)'] for p in team)
        skill_mean = np.mean([p['Skill'] for p in team])
        team_means.append(skill_mean)
        all_players.update(names)
        
        print(f"  Equipa {j+1}: {positions} | Skill Médio: {skill_mean:.2f} | Orçamento: {salary}M")
        
        if not is_valid_team(team):
            print("    ⚠️ Equipa inválida!")
            valid = False

    if len(all_players) < 35:
        print("    ⚠️ Jogadores repetidos entre equipas!")
        valid = False

    print(f"  ➤ Fitness (desvio padrão): {fitness(indiv):.4f}")
    print("  ✅ Indivíduo válido" if valid else "  ❌ Indivíduo inválido")



Indivíduo 1
  Equipa 1: ['GK', 'DEF', 'DEF', 'MID', 'MID', 'FWD', 'FWD'] | Skill Médio: 84.00 | Orçamento: 594M
  Equipa 2: ['GK', 'DEF', 'DEF', 'MID', 'MID', 'FWD', 'FWD'] | Skill Médio: 86.86 | Orçamento: 700M
  Equipa 3: ['GK', 'DEF', 'DEF', 'MID', 'MID', 'FWD', 'FWD'] | Skill Médio: 87.57 | Orçamento: 705M
  Equipa 4: ['GK', 'DEF', 'DEF', 'MID', 'MID', 'FWD', 'FWD'] | Skill Médio: 86.57 | Orçamento: 700M
  Equipa 5: ['GK', 'DEF', 'DEF', 'MID', 'MID', 'FWD', 'FWD'] | Skill Médio: 87.00 | Orçamento: 725M
  ➤ Fitness (desvio padrão): 1.2434
  ✅ Indivíduo válido

Indivíduo 2
  Equipa 1: ['GK', 'DEF', 'DEF', 'MID', 'MID', 'FWD', 'FWD'] | Skill Médio: 87.29 | Orçamento: 700M
  Equipa 2: ['GK', 'DEF', 'DEF', 'MID', 'MID', 'FWD', 'FWD'] | Skill Médio: 85.86 | Orçamento: 647M
  Equipa 3: ['GK', 'DEF', 'DEF', 'MID', 'MID', 'FWD', 'FWD'] | Skill Médio: 88.00 | Orçamento: 740M
  Equipa 4: ['GK', 'DEF', 'DEF', 'MID', 'MID', 'FWD', 'FWD'] | Skill Médio: 84.86 | Orçamento: 655M
  Equipa 5: ['GK'

In [14]:
print("Total de jogadores:", len(df))
print("Distribuição por posição:")
print(df['Position'].value_counts())


Total de jogadores: 35
Distribuição por posição:
Position
DEF    10
MID    10
FWD    10
GK      5
Name: count, dtype: int64


In [15]:
team, used_ids = create_valid_team(players_by_position)
if team:
    print("Equipa gerada com sucesso!")
    for player in team:
        print(f"{player['Name']} - {player['Position']} - Skill: {player['Skill']} - Salary: {player['Salary (€M)']}M")
else:
    print("Erro: Não foi possível gerar uma equipa válida.")


Equipa gerada com sucesso!
Jordan Smith - GK - Skill: 88 - Salary: 100M
Jaxon Griffin - DEF - Skill: 79 - Salary: 65M
Daniel Foster - DEF - Skill: 90 - Salary: 110M
Austin Torres - MID - Skill: 82 - Salary: 80M
Connor Hayes - MID - Skill: 89 - Salary: 105M
Xavier Bryant - FWD - Skill: 90 - Salary: 120M
Julian Scott - FWD - Skill: 92 - Salary: 130M


In [16]:
indiv = create_individual()
if indiv:
    print("Indivíduo gerado com sucesso! Liga com 5 equipas:")
    for i, team in enumerate(indiv):
        print(f"\nEquipa {i+1}:")
        for p in team:
            print(f"  {p['Name']} ({p['Position']}) - Skill: {p['Skill']} - €{p['Salary (€M)']}M")
else:
    print("Erro: Não foi possível gerar um indivíduo válido.")


Indivíduo gerado com sucesso! Liga com 5 equipas:

Equipa 1:
  Alex Carter (GK) - Skill: 85 - €90M
  Jaxon Griffin (DEF) - Skill: 79 - €65M
  Daniel Foster (DEF) - Skill: 90 - €110M
  Spencer Ward (MID) - Skill: 84 - €85M
  Austin Torres (MID) - Skill: 82 - €80M
  Julian Scott (FWD) - Skill: 92 - €130M
  Xavier Bryant (FWD) - Skill: 90 - €120M

Equipa 2:
  Ryan Mitchell (GK) - Skill: 83 - €85M
  Ethan Howard (DEF) - Skill: 80 - €70M
  Owen Parker (DEF) - Skill: 88 - €100M
  Dylan Morgan (MID) - Skill: 91 - €115M
  Gavin Richardson (MID) - Skill: 87 - €95M
  Elijah Sanders (FWD) - Skill: 93 - €140M
  Zachary Nelson (FWD) - Skill: 86 - €92M

Equipa 3:
  Chris Thompson (GK) - Skill: 80 - €80M
  Brayden Hughes (DEF) - Skill: 87 - €100M
  Lucas Bennett (DEF) - Skill: 85 - €90M
  Connor Hayes (MID) - Skill: 89 - €105M
  Hunter Cooper (MID) - Skill: 83 - €85M
  Landon Powell (FWD) - Skill: 89 - €110M
  Chase Murphy (FWD) - Skill: 86 - €95M

Equipa 4:
  Jordan Smith (GK) - Skill: 88 - €100M
  